In [20]:
import numpy as np
import time

A = np.arange(12)

start = time.time()
for i in range(10000):
    B = np.roll(A, 2) + np.roll(A, -2)
    B[:2], B[-2:] = np.zeros(2), np.zeros(2)
print(B)
end = time.time()
print("roll time: ", end-start)


start = time.time()
for i in range(10000):
    B = np.zeros(12)
    B[2:-2] = A[4:] + A[:-4]
print(B)
end = time.time()
print(f"slice time: {end-start:0.6f}")


A = np.arange(100).reshape(4,25)
B = -24*np.roll(A, -5, 1) + 24*np.roll(A, 5, 1)
C = (-A[:, 11:-1] + A[:, 1:-11])*24
print(B)
print(C)

# fun = np.arange(100).reshape(4,25)
# D = - np.roll(fun,-6,1) + 24*np.roll(fun,-5,1) - 192*np.roll(fun,-4,1) + np.roll(fun,+6,1) - 24*np.roll(fun,+5,1) + 192*np.roll(fun,+4,1)
# E = (-np.roll(fun,-6,1)+ np.roll(fun,+6,1)) +\
# (np.roll(fun,-5,1) -np.roll(fun,+5,1))*24 +\
# (-np.roll(fun,-4,1) + np.roll(fun,4,1))*192


#print(D, E)
   

#roll can be ten times more time consuming!!!
#so it is worth to change derivative function!

[ 0  0  4  6  8 10 12 14 16 18  0  0]
roll time:  0.3899204730987549
[ 0.  0.  4.  6.  8. 10. 12. 14. 16. 18.  0.  0.]
slice time: 0.024933
[[ 360  360  360  360  360 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240  360  360  360  360  360]
 [ 360  360  360  360  360 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240  360  360  360  360  360]
 [ 360  360  360  360  360 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240  360  360  360  360  360]
 [ 360  360  360  360  360 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240  360  360  360  360  360]]
[[-240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240]
 [-240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240]
 [-240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240]
 [-240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240 -240]]


In [3]:
import numpy as np
np.set_printoptions(suppress=True, precision=4, linewidth= 150 )

def dev3(x, fun):      #this is the function resposible for most of the computational time (currently about a quarter of tot time)
    """Numerical approximation for the third derivative"""                     #axis=1 because it's roling 2D array along rows
    #x, fun, derivative = np.array(x), np.array(fun), np.zeros(fun.shape)
    derivative = - np.roll(fun,-6,1) + 24*np.roll(fun,-5,1) - 192*np.roll(fun,-4,1) + 488*np.roll(fun,-3,1) + 387*np.roll(fun,-2,1) - 1584*np.roll(fun,-1,1) \
                 + np.roll(fun,+6,1) - 24*np.roll(fun,+5,1) + 192*np.roll(fun,+4,1) - 488*np.roll(fun,+3,1) - 387*np.roll(fun,+2,1) + 1584*np.roll(fun,+1,1)
    derivative /= np.power(np.roll(x,-1,1) - x, 3) 
    derivative /= 12**3
    derivative[:,:6], derivative[:,-6:] = np.zeros((len(x),6)), np.zeros((len(x),6))  #need to correct for beggining and end of array because of the rolls
    return derivative

def new_dev3(x, fun):
    
    k6 = ( - fun[:, 12:  ] + fun[:,  :-12] ) * 1
    k5 = ( + fun[:, 11:-1] - fun[:, 1:-11] ) * 24
    k4 = ( - fun[:, 10:-2] + fun[:, 2:-10] ) * 192
    k3 = ( + fun[:,  9:-3] - fun[:, 3:-9 ] ) * 488
    k2 = ( + fun[:,  8:-4] - fun[:, 4:-8 ] ) * 387
    k1 = ( - fun[:,  7:-5] + fun[:, 5:-7 ] ) * 1584
    
    dev = k1 + k2 + k3 + k4 + k5 + k6
    dev /= np.power( x[:, 7:-5] - x[:, 6:-6], 3)
    dev /= 12**3
    
    derivative = np.zeros(fun.shape)
    derivative[:, 6:-6] = dev                   #need to pad with zeros left and right to return array with same shape
    return derivative
    
    
def test_dev(x, fun):
    
    res1 = dev3(x, fun)
    res2 = new_dev3(x, fun)
    
    if np.allclose(res1, res2):
        print("The two functions give the same result!")
    else:
        print("Something is wrong, functions behave differently!")
        print(res1, "\n", res2)
        
A = np.arange(-50, 50, dtype="float").reshape((4,25))
B = A**4 + 3*A**3 + 5*A**2 + 8*A + 13
#print(A[:, :10], B[:, :10])
test_dev(A, B)

The two functions give the same result!


In [4]:
def edited_fun_derivative3(x, fun):
    """Numerical approximation for the third derivative"""   
    x, fun, derivative = np.array(x), np.array(fun), np.zeros(len(fun))
    derivative += - np.roll(fun,-6) + 24*np.roll(fun,-5) - 192*np.roll(fun,-4) + 488*np.roll(fun,-3) + 387*np.roll(fun,-2) - 1584*np.roll(fun,-1)
    derivative += + np.roll(fun,+6) - 24*np.roll(fun,+5) + 192*np.roll(fun,+4) - 488*np.roll(fun,+3) - 387*np.roll(fun,+2) + 1584*np.roll(fun,+1)
    derivative /= np.power(np.roll(x,-1) - x, 3)
    derivative /= 12**3
    derivative[:6], derivative[-6:] = np.zeros(6), np.zeros(6)  #need to correct for beggining and end of array
    return derivative
    
def fun(x):
    return np.power(x, 3) #+ 2*np.power(x, 2) + 3*np.power(x, 4)
    
x = range(-10, 10)
fun = fun(x)
print(edited_fun_derivative3(x, fun))

[0. 0. 0. 0. 0. 0. 6. 6. 6. 6. 6. 6. 6. 6. 0. 0. 0. 0. 0. 0.]


In [4]:
import numpy as np

C = np.arange(12).reshape((4,3))
print(np.roll(C, 1, 1))
print(C.T)



[[ 2  0  1]
 [ 5  3  4]
 [ 8  6  7]
 [11  9 10]]
[[ 0  3  6  9]
 [ 1  4  7 10]
 [ 2  5  8 11]]


In [18]:
import numpy as np

def reshape_yspace(all_y_spaces):     #starting shape (4, 132, 144)
    A = all_y_spaces
    reshaped = np.zeros( (len(A[0]), len(A), len(A[0,0])) )
    for n in range(len(A[0])):
        for i in range(len(A)):
            reshaped[n, i, :] = A[i, n, :]
    return reshaped

def new_reshape(A):
    return np.stack(np.split(A, 4, axis=0), axis=2)[0]

A = np.arange(60).reshape((4, 5, 3))

def test_fun(A):
    res1 = reshape_yspace(A)
    res2 = new_reshape(A)
    
    if res1.shape != res2.shape:
        print("shapes are different!")
        
    if ~np.allclose(res1, res2):
        print("arrays are not the same!")
        print(res1, "\n", res2)
        
test_fun(A)

arrays are not the same!
[[[ 0.  1.  2.]
  [15. 16. 17.]
  [30. 31. 32.]
  [45. 46. 47.]]

 [[ 3.  4.  5.]
  [18. 19. 20.]
  [33. 34. 35.]
  [48. 49. 50.]]

 [[ 6.  7.  8.]
  [21. 22. 23.]
  [36. 37. 38.]
  [51. 52. 53.]]

 [[ 9. 10. 11.]
  [24. 25. 26.]
  [39. 40. 41.]
  [54. 55. 56.]]

 [[12. 13. 14.]
  [27. 28. 29.]
  [42. 43. 44.]
  [57. 58. 59.]]] 
 [[[ 0  1  2]
  [15 16 17]
  [30 31 32]
  [45 46 47]]

 [[ 3  4  5]
  [18 19 20]
  [33 34 35]
  [48 49 50]]

 [[ 6  7  8]
  [21 22 23]
  [36 37 38]
  [51 52 53]]

 [[ 9 10 11]
  [24 25 26]
  [39 40 41]
  [54 55 56]]

 [[12 13 14]
  [27 28 29]
  [42 43 44]
  [57 58 59]]]
